# Import required libraries

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

import shutil
import random

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Flatten, Input, BatchNormalization, Dense, Activation, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder

from copy import deepcopy

2024-11-21 08:05:45.794226: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-21 08:05:46.758058: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Federated Proximal Algorithm

In [2]:
def loss_classifier(predictions, labels):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=predictions)
    return tf.reduce_mean(loss)


def loss_dataset(model, dataset, loss_f):
    loss = 0
    for idx, (features, labels) in enumerate(dataset):
        # Convert PyTorch tensors to NumPy and then to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        predictions = model(features_tf)
        loss += loss_f(predictions, labels_tf)
    loss /= (idx + 1)
    return loss


def accuracy_dataset(model, dataset):
    correct = 0
    total = 0
    for features, labels in dataset:
        # Convert PyTorch tensors to NumPy and then to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        predictions = model(features_tf)
        predicted = tf.argmax(predictions, axis=1, output_type=tf.int32)
        correct += tf.reduce_sum(tf.cast(tf.equal(predicted, labels_tf), tf.int32)).numpy()
        total += labels_tf.shape[0]

    accuracy = 100 * correct / total
    return accuracy


def train_step(model, model_0, mu, optimizer, train_data, loss_f):
    total_loss = 0
    for idx, (features, labels) in enumerate(train_data):
        # Convert PyTorch tensors to NumPy and then to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        with tf.GradientTape() as tape:
            predictions = model(features_tf)
            loss = loss_f(predictions, labels_tf)
            loss += mu / 2 * difference_models_norm_2(model, model_0)

        total_loss += loss
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return total_loss / (idx + 1)


def local_learning(model, mu, optimizer, train_data, epochs, loss_f):
    # Clone the model instead of using deepcopy
    model_0 = tf.keras.models.clone_model(model)
    model_0.set_weights(model.get_weights())  # Set weights to be identical initially

    for e in range(epochs):
        local_loss = train_step(model, model_0, mu, optimizer, train_data, loss_f)

    return local_loss


def difference_models_norm_2(model_1, model_2):
    norm = tf.reduce_sum([tf.reduce_sum(tf.square(w1 - w2)) for w1, w2 in zip(model_1.trainable_variables, model_2.trainable_variables)])
    return norm

def set_to_zero_model_weights(model):
    for layer_weights in model.trainable_variables:
        layer_weights.assign(tf.zeros_like(layer_weights))

def average_models(model, clients_models_hist, weights):
    set_to_zero_model_weights(model)
    for k, client_hist in enumerate(clients_models_hist):
        for idx, layer_weights in enumerate(model.trainable_variables):
            contribution = client_hist[idx] * weights[k]
            layer_weights.assign_add(contribution)

#_______________________________________________________

In [3]:
def FedProx(model, training_sets, n_iter, testing_sets, mu=0, epochs=5, lr=0.01, decay=1):
    # Verify that `model` is a Keras model instance
    if not isinstance(model, tf.keras.Model):
        raise TypeError("The provided model is not a TensorFlow Keras model. Please provide a valid Keras model.")

    loss_f = loss_classifier
    K = len(training_sets)
    n_samples = sum([len(db) for db in training_sets])
    weights = [len(db) / n_samples for db in training_sets]
    print("Clients' weights:", weights)

    # Initialize history lists for training and testing
    train_loss_hist = []
    train_acc_hist = []
    test_loss_hist = []
    test_acc_hist = []
    models_hist = []

    for i in range(n_iter):
        clients_params = []
        clients_losses = []
        clients_accuracies = []

        for k in range(K):
            # Clone the model and set weights for local training
            local_model = tf.keras.models.clone_model(model)
            local_model.set_weights(model.get_weights())

            local_optimizer = tf.keras.optimizers.SGD(learning_rate=lr)

            # Perform local training and track the loss
            local_loss = local_learning(local_model, mu, local_optimizer, training_sets[k], epochs, loss_f)
            clients_losses.append(local_loss)

            # Track training accuracy for the client
            train_acc = accuracy_dataset(local_model, training_sets[k])
            clients_accuracies.append(train_acc)

            # Store model parameters (deep copy to ensure immutability)
            clients_params.append([tf.identity(tens_param) for tens_param in local_model.trainable_variables])

        # Average the local models into the global model
        average_models(model, clients_params, weights=weights)
        models_hist.append(deepcopy(clients_params))

        # Collect metrics for this iteration
        train_loss_hist.append(clients_losses)
        train_acc_hist.append(clients_accuracies)

        # Compute testing metrics using the global model
        test_loss_hist.append([loss_dataset(model, dl, loss_f).numpy() for dl in testing_sets])
        test_acc_hist.append([accuracy_dataset(model, dl) for dl in testing_sets])

        # Update learning rate by decay factor
        lr *= decay
        print(f'====> i: {i+1} Server Test Accuracy: {test_acc_hist[-1]}')

    return model, train_loss_hist, train_acc_hist, test_loss_hist, test_acc_hist



# Plot results + Save results in text file

In [4]:
import matplotlib.pyplot as plt

def plot_acc_loss(title: str, train_loss_hist: list, train_acc_hist: list,
                  test_loss_hist: list, test_acc_hist: list):
    plt.figure(figsize=(15, 5))  # Make the plot wider

    # Plot training loss
    plt.subplot(2, 2, 1)
    lines = plt.plot(train_loss_hist)
    plt.title("Training Loss")
    plt.legend(lines, [f"C{i+1}" for i in range(len(train_loss_hist[0]))])

    # Plot training accuracy
    plt.subplot(2, 2, 2)
    lines = plt.plot(train_acc_hist)
    plt.title("Training Accuracy")
    plt.legend(lines, [f"C{i+1}" for i in range(len(train_acc_hist[0]))])

    # Plot testing loss
    plt.subplot(2, 2, 3)
    lines = plt.plot(test_loss_hist)
    plt.title("Testing Loss")
    plt.legend(lines, [f"C{i+1}" for i in range(len(test_loss_hist[0]))])

    # Plot testing accuracy
    plt.subplot(2, 2, 4)
    lines = plt.plot(test_acc_hist)
    plt.title("Testing Accuracy")
    plt.legend(lines, [f"C{i+1}" for i in range(len(test_acc_hist[0]))])

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()


def save_history_to_file(filename, n_iter, train_loss_hist, train_acc_hist,
                         test_loss_hist, test_acc_hist):
    with open(filename, 'w') as f:
        f.write("FedProx Training Results\n")
        f.write("="*50 + "\n")
        f.write(f"Number of iterations: {n_iter}\n\n")

        # Write the history for each iteration
        for i in range(n_iter):
            f.write(f"Iteration {i+1}:\n")
            f.write(f"Train Loss: {train_loss_hist[i]}\n")
            f.write(f"Train Accuracy: {train_acc_hist[i]}\n")
            f.write(f"Test Loss: {test_loss_hist[i]}\n")
            f.write(f"Test Accuracy: {test_acc_hist[i]}\n")
            f.write("-"*50 + "\n")

    print(f"Training history saved to {filename}")

# Load training and testing data

In [5]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Directories for each client
split_dirs = [
    "Federated_Learning_NON_IID/Model_1",
    "Federated_Learning_NON_IID/Model_2",
    "Federated_Learning_NON_IID/Model_3",
    "Federated_Learning_NON_IID/Model_4"
]

# Number of clients
n_clients = len(split_dirs)

# Updated image transformations: Normalize first, then permute
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert image to tensor format (C, H, W)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    transforms.Lambda(lambda x: x.permute(1, 2, 0))  # Permute to (H, W, C)
])

def get_tumor_dataloaders(split_dirs, batch_size=25, shuffle=True):
    """
    Returns data loaders for all clients for both training and testing sets.
    """
    tumor_iid_train_dls = []
    tumor_iid_test_dls = []

    for client_idx, client_dir in enumerate(split_dirs):
        # Get the directory for the current client
        train_dir = os.path.join(client_dir, 'train')
        test_dir = os.path.join(client_dir, 'test')

        # Check if the directories exist
        if not os.path.exists(train_dir) or not os.path.exists(test_dir):
            print(f"Directory not found for client {client_idx + 1}:")
            print(f"Train dir: {train_dir}")
            print(f"Test dir: {test_dir}")
            continue

        # Load training data for the current client
        train_dataset = datasets.ImageFolder(train_dir, transform=transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

        # Load testing data for the current client
        test_dataset = datasets.ImageFolder(test_dir, transform=transform)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Append the dataloaders for the current client to the list
        tumor_iid_train_dls.append(train_loader)
        tumor_iid_test_dls.append(test_loader)

    return tumor_iid_train_dls, tumor_iid_test_dls

# Get the training and testing data loaders
Tumor_iid_train_dls, Tumor_iid_test_dls = get_tumor_dataloaders(split_dirs, batch_size=25)

# Checking the sizes of the images in the data loaders to verify the shape
for batch_idx, (images, labels) in enumerate(Tumor_iid_train_dls[0]):  # Checking for client 1
    print(f"Batch {batch_idx} image sizes: {images.size()}")  # Should print torch.Size([25, 224, 224, 3])
    break  # Check only the first batch


Batch 0 image sizes: torch.Size([25, 224, 224, 3])


# Note : 
### firest ResNet model trained for 20 rounds and saved that model
### Again train the saved model for 30 rounds.
### remaining 50 rounds will be implemnted here

# Implementation FL for (50 to 100) rounds

In [6]:
# Initial parameters
MODEL_PATH = 'global_model_keras/global_model_rounds_80.h5'
n_iter = 100
rounds_per_segment = 10
mu = 0.4
epochs = 1
lr = 0.001

# Initialize lists to store cumulative training and testing metrics
all_train_loss = []
all_train_acc = []
all_test_loss = []
all_test_acc = []
# Load the initial model
model = tf.keras.models.load_model(MODEL_PATH)

# Function for FedProx training in segments
for i in range(80, n_iter, rounds_per_segment):
    print(f"Starting rounds {i+1} to {i+rounds_per_segment}")
    
    # Train for the specified number of rounds in this segment
    model, train_loss_f, train_acc_f, test_loss_f, test_acc_f = FedProx(
        model, Tumor_iid_train_dls, rounds_per_segment, Tumor_iid_test_dls, mu=mu, epochs=epochs, lr=lr
    )
    
    # Append the metrics to the cumulative lists
    all_train_loss.extend(train_loss_f)
    all_train_acc.extend(train_acc_f)
    all_test_loss.extend(test_loss_f)
    all_test_acc.extend(test_acc_f)
    
    # Save the model at the end of each segment
    segment_model_path = f'{MODEL_PATH}global_model_rounds_{i+rounds_per_segment}.h5'
    model.save(segment_model_path)
    print(f"Model saved at: {segment_model_path}")

    # Reload the model to ensure continuity for the next segment
    model = tf.keras.models.load_model(segment_model_path)

    

2024-11-17 17:46:30.494352: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30302 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:89:00.0, compute capability: 7.0


Starting rounds 81 to 90
Clients' weights: [0.22620689655172413, 0.2303448275862069, 0.2868965517241379, 0.25655172413793104]


2024-11-17 17:46:37.645156: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2024-11-17 17:46:41.204983: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x2264dcd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-11-17 17:46:41.205021: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-11-17 17:46:41.287144: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


====> i: 1 Server Test Accuracy: [75.0, 74.52380952380952, 78.92156862745098, 87.83333333333333]
====> i: 2 Server Test Accuracy: [73.57142857142857, 72.73809523809524, 79.6078431372549, 85.33333333333333]
====> i: 3 Server Test Accuracy: [75.83333333333333, 74.64285714285714, 80.88235294117646, 88.16666666666667]
====> i: 4 Server Test Accuracy: [76.07142857142857, 75.11904761904762, 81.96078431372548, 87.0]
====> i: 5 Server Test Accuracy: [76.30952380952381, 74.76190476190476, 80.98039215686275, 90.33333333333333]
====> i: 6 Server Test Accuracy: [76.42857142857143, 74.88095238095238, 81.27450980392157, 89.33333333333333]
====> i: 7 Server Test Accuracy: [75.5952380952381, 75.0, 78.13725490196079, 89.5]
====> i: 8 Server Test Accuracy: [75.47619047619048, 74.64285714285714, 78.92156862745098, 89.33333333333333]
====> i: 9 Server Test Accuracy: [73.92857142857143, 74.4047619047619, 78.62745098039215, 87.16666666666667]
====> i: 10 Server Test Accuracy: [76.19047619047619, 75.0, 78.92

# Save the results in text file

In [7]:
# Save the cumulative history to a file after all rounds are complete
train_loss = all_train_loss[-20:]  # Adjusted to reflect 30 rounds for rounds 51 to 80
train_acc = all_train_acc[-20:]
test_loss = all_test_loss[-20:]
test_acc = all_test_acc[-20:]

# Save the history to a file
def save_history_to_file(filename, train_loss, train_acc, test_loss, test_acc):
    with open(filename, "w") as f:
        f.write("FedProx Training and Testing Metrics\n")
        f.write("=" * 50 + "\n")
        
        for i in range(len(train_loss)):
            try:
                train_loss_val = float(train_loss[i][0]) if isinstance(train_loss[i], list) else float(train_loss[i])
                train_acc_val = float(train_acc[i][0]) if isinstance(train_acc[i], list) else float(train_acc[i])
                test_loss_val = float(test_loss[i][0]) if isinstance(test_loss[i], list) else float(test_loss[i])
                test_acc_val = float(test_acc[i][0]) if isinstance(test_acc[i], list) else float(test_acc[i])
                
                f.write(f"Round {i + 81}:\n")  # Adjusted to start from round 51
                f.write(f"Train Loss: {train_loss_val:.4f}\n")
                f.write(f"Train Accuracy: {train_acc_val:.4f}\n")
                f.write(f"Test Loss: {test_loss_val:.4f}\n")
                f.write(f"Test Accuracy: {test_acc_val:.4f}\n")
                f.write("-" * 50 + "\n")
                
            except (ValueError, TypeError) as e:
                f.write(f"Error processing round {i + 51}: {e}\n")
                f.write("-" * 50 + "\n")
    
    print(f"History saved to {filename}")

# File to save the results
output_file = "results-mu=0.4_81_to_100_rounds.txt"

# Save the data to the file
save_history_to_file(output_file, train_loss, train_acc, test_loss, test_acc)


History saved to results-mu=0.4_81_to_100_rounds.txt


In [8]:
# Save the cumulative history to a file after all rounds are complete
train_loss = all_train_loss[-20:]  # Adjusted to reflect 30 rounds for rounds 51 to 80
train_acc = all_train_acc[-20:]
test_loss = all_test_loss[-20:]
test_acc = all_test_acc[-20:]

# Save the history to a file with detailed formatting
def save_history_to_file(filename, train_loss, train_acc, test_loss, test_acc):
    with open(filename, "w") as f:
        f.write("FedProx Training and Testing Metrics\n")
        f.write("=" * 50 + "\n")
        
        for i in range(len(train_loss)):
            try:
                # Ensure data is printed in the format shown, converting Tensors to their numpy values if needed
                train_loss_str = [float(tensor.numpy()) if hasattr(tensor, 'numpy') else float(tensor) for tensor in train_loss[i]]
                train_acc_str = [float(val) for val in train_acc[i]]
                test_loss_str = [float(val) for val in test_loss[i]]
                test_acc_str = [float(val) for val in test_acc[i]]
                
                f.write(f"Iteration {i + 81}:\n")
                f.write(f"Train Loss: {train_loss_str}\n")
                f.write(f"Train Accuracy: {train_acc_str}\n")
                f.write(f"Test Loss: {test_loss_str}\n")
                f.write(f"Test Accuracy: {test_acc_str}\n")
                f.write("-" * 50 + "\n")
                
            except (ValueError, TypeError) as e:
                f.write(f"Error processing round {i + 51}: {e}\n")
                f.write("-" * 50 + "\n")
    
    print(f"History saved to {filename}")

# File to save the results
output_file = "results-mu=0.4_81_to_100.txt"

# Save the data to the file
save_history_to_file(output_file, train_loss, train_acc, test_loss, test_acc)


History saved to results-mu=0.4_81_to_100.txt


In [6]:
MODEL_PATH = 'global_model_keras/'
model = tf.keras.models.load_model(MODEL_PATH)

2024-11-18 14:26:16.492603: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20716 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:89:00.0, compute capability: 7.0


In [7]:
n_iter = 15
# Execute FedProx with the new 4-client setup
model_f, train_loss_f, train_acc_f, test_loss_f, test_acc_f = FedProx(
    model, Tumor_iid_train_dls, n_iter, Tumor_iid_test_dls, mu=0.4, epochs=1, lr=0.001
)

Clients' weights: [0.22620689655172413, 0.2303448275862069, 0.2868965517241379, 0.25655172413793104]


2024-11-18 14:26:28.094125: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2024-11-18 14:26:31.161960: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x2ca14ed0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-11-18 14:26:31.161987: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-11-18 14:26:31.234761: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


====> i: 1 Server Test Accuracy: [67.02380952380952, 58.69047619047619, 68.72549019607843, 78.33333333333333]
====> i: 2 Server Test Accuracy: [69.64285714285714, 58.214285714285715, 65.98039215686275, 80.0]
====> i: 3 Server Test Accuracy: [69.76190476190476, 63.57142857142857, 72.6470588235294, 82.5]
====> i: 4 Server Test Accuracy: [72.61904761904762, 64.52380952380952, 73.33333333333333, 85.33333333333333]
====> i: 5 Server Test Accuracy: [72.5, 64.28571428571429, 72.3529411764706, 84.33333333333333]
====> i: 6 Server Test Accuracy: [65.95238095238095, 64.88095238095238, 74.50980392156863, 79.0]
====> i: 7 Server Test Accuracy: [74.88095238095238, 66.54761904761905, 80.3921568627451, 86.66666666666667]
====> i: 8 Server Test Accuracy: [68.33333333333333, 62.142857142857146, 71.76470588235294, 80.0]
====> i: 9 Server Test Accuracy: [72.26190476190476, 62.976190476190474, 72.54901960784314, 81.83333333333333]
====> i: 10 Server Test Accuracy: [75.5952380952381, 67.26190476190476, 76.

In [9]:
# Save the cumulative history to a file after all rounds are complete
train_loss = train_loss_f[-15:]  # Adjusted to reflect 30 rounds for rounds 51 to 80
train_acc = train_acc_f[-15:]
test_loss = test_loss_f[-15:]
test_acc = test_acc_f[-15:]

# Save the history to a file with detailed formatting
def save_history_to_file(filename, train_loss, train_acc, test_loss, test_acc):
    with open(filename, "w") as f:
        f.write("FedProx Training and Testing Metrics\n")
        f.write("=" * 50 + "\n")
        
        for i in range(len(train_loss)):
            try:
                # Ensure data is printed in the format shown, converting Tensors to their numpy values if needed
                train_loss_str = [float(tensor.numpy()) if hasattr(tensor, 'numpy') else float(tensor) for tensor in train_loss[i]]
                train_acc_str = [float(val) for val in train_acc[i]]
                test_loss_str = [float(val) for val in test_loss[i]]
                test_acc_str = [float(val) for val in test_acc[i]]
                
                f.write(f"Iteration {i + 1}:\n")
                f.write(f"Train Loss: {train_loss_str}\n")
                f.write(f"Train Accuracy: {train_acc_str}\n")
                f.write(f"Test Loss: {test_loss_str}\n")
                f.write(f"Test Accuracy: {test_acc_str}\n")
                f.write("-" * 50 + "\n")
                
            except (ValueError, TypeError) as e:
                f.write(f"Error processing round {i + 51}: {e}\n")
                f.write("-" * 50 + "\n")
    
    print(f"History saved to {filename}")

# File to save the results
output_file = "results-mu=0.4_1_to_15.txt"

# Save the data to the file
save_history_to_file(output_file, train_loss, train_acc, test_loss, test_acc)


History saved to results-mu=0.4_1_to_15.txt


In [10]:
# Save the model at the end of each segment
segment_model_path = f'{MODEL_PATH}global_model_rounds_1_to_15.h5'
model_f.save(segment_model_path)
print(f"Model saved at: {segment_model_path}")

Model saved at: global_model_keras/global_model_rounds_1_to_15.h5


In [11]:
print("__________________Training complete and results saved__________________")

__________________Training complete and results saved__________________


In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import tensorflow as tf
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score, classification_report
import numpy as np
import os

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert image to tensor format (C, H, W)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    transforms.Lambda(lambda x: x.permute(1, 2, 0))  # Permute to (H, W, C)
])

# Load the testing dataset
test_dir = 'BrainTumor_MRI/Testing'
test_dataset = datasets.ImageFolder(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Class names mapping (adjust if necessary)
class_names = test_dataset.classes

In [14]:
# Define class names
class_names = ["glioma", "meningioma", "notumor", "pituitary"]  # Modify according to your classes

# Function to evaluate the global model on the testing dataset
def evaluate_global_model(model, test_loader, output_file="Global_Model_Results.txt"):
    import os

    all_preds = []
    all_labels = []
    total_loss = 0

    for features, labels in test_loader:
        # Convert to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        # Get predictions
        predictions = model(features_tf)
        predicted = tf.argmax(predictions, axis=1, output_type=tf.int32)

        # Accumulate predictions and labels
        all_preds.extend(predicted.numpy())
        all_labels.extend(labels_np)

        # Compute loss
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_tf, logits=predictions)
        total_loss += tf.reduce_mean(loss).numpy()

    # Compute metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_preds)

    # Classification report
    classification_report_str = classification_report(all_labels, all_preds, target_names=class_names)

    # Append metrics to a file
    with open(output_file, "a") as file:
        file.write("Evaluation Metrics:\n")
        file.write(f"Accuracy: {accuracy * 100:.2f}%\n")
        file.write(f"Loss: {total_loss / len(test_loader):.4f}\n")
        file.write(f"Precision: {precision:.4f}\n")
        file.write(f"Recall: {recall:.4f}\n")
        file.write(f"F1 Score: {f1:.4f}\n\n")
        file.write("Classification Report:\n")
        file.write(classification_report_str)
        file.write("\nConfusion Matrix:\n")
        file.write("\n".join(["\t".join(map(str, row)) for row in conf_matrix]))
        file.write("\n" + "-" * 50 + "\n")

    # Print metrics
    print("\nClassification Report:\n", classification_report_str)
    print(f"\nAccuracy: {accuracy * 100:.2f}%")
    print(f"Loss: {total_loss / len(test_loader):.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    # Plot and save heatmap
    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.savefig('confusion_matrix.png')  # Save as PNG
    plt.close()  # Close the figure


In [15]:
MODEL_PATH = 'global_model_keras/global_model_rounds_100.h5'
model = tf.keras.models.load_model(MODEL_PATH)

# model_names = ["global_model_keras/global_model_rounds_20.h5",
#                "global_model_keras/global_model_rounds_60.h5",
#                "global_model_keras/global_model_rounds_80.h5",
#                "global_model_keras/global_model_rounds_100.h5"]


In [16]:
# Evaluate the model on the testing dataset
evaluate_global_model(model, test_loader)


Classification Report:
               precision    recall  f1-score   support

      glioma       0.88      0.88      0.88       300
  meningioma       0.87      0.82      0.84       306
     notumor       0.98      0.98      0.98       405
   pituitary       0.91      0.96      0.93       300

    accuracy                           0.91      1311
   macro avg       0.91      0.91      0.91      1311
weighted avg       0.91      0.91      0.91      1311


Accuracy: 91.46%
Loss: 0.8288
Precision: 0.9141
Recall: 0.9146
F1 Score: 0.9141


In [13]:
import matplotlib.pyplot as plt

# Data: Metrics for different rounds (example values, replace with actual data)
rounds = [20, 60, 80, 100]
accuracies = [87.49, 90.69, 90.92, 91.46]
losses = [0.8670, 0.8367, 0.8369, 0.8288]
recalls = [87.49, 90.69, 90.92, 91.46]  # Same as accuracies since recall = accuracy in weighted average here
f1_scores = [87.19, 90.66, 90.88, 91.41]

# Plot Accuracy
plt.figure(figsize=(10, 5))
plt.plot(rounds, accuracies, marker='o', linestyle='-', color='b', label='Accuracy')
plt.title('Accuracy of global model with mu=0.4')
plt.xlabel('Rounds')
plt.ylabel('Accuracy (%)')
plt.xticks(rounds)
plt.grid(True)
plt.legend()
plt.savefig('accuracy_over_rounds.png')  # Save plot as PNG
plt.close()  # Close the figure to avoid overlap

# Plot Loss
plt.figure(figsize=(10, 5))
plt.plot(rounds, losses, marker='o', linestyle='-', color='r', label='Loss')
plt.title('Loss of global model with mu=0.4')
plt.xlabel('Rounds')
plt.ylabel('Loss')
plt.xticks(rounds)
plt.grid(True)
plt.legend()
plt.savefig('loss_over_rounds.png')  # Save plot as PNG
plt.close()

# Plot Recall
plt.figure(figsize=(10, 5))
plt.plot(rounds, recalls, marker='o', linestyle='-', color='g', label='Recall')
plt.title('Recall of global model with mu=0.4')
plt.xlabel('Rounds')
plt.ylabel('Recall (%)')
plt.xticks(rounds)
plt.grid(True)
plt.legend()
plt.savefig('recall_over_rounds.png')  # Save plot as PNG
plt.close()

# Plot F1 Score
plt.figure(figsize=(10, 5))
plt.plot(rounds, f1_scores, marker='o', linestyle='-', color='m', label='F1 Score')
plt.title('F1 of global model with mu=0.4')
plt.xlabel('Rounds')
plt.ylabel('F1 Score (%)')
plt.xticks(rounds)
plt.grid(True)
plt.legend()
plt.savefig('f1_score_over_rounds.png')  # Save plot as PNG
plt.close()


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg19 import preprocess_input
import numpy as np

# Define class names
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Function to predict the class of a single image
def predict_image(image_path, model):
    # Load and preprocess the image
    image = load_img(image_path, target_size=(224, 224))  # Resize to 224x224
    image_array = img_to_array(image)  # Convert to numpy array
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    image_array = preprocess_input(image_array)  # Normalize for VGG19

    # Perform inference
    predictions = model(image_array, training=False)  # Inference mode
    probabilities = tf.nn.softmax(predictions[0]).numpy()  # Apply softmax
    predicted_class = np.argmax(probabilities)

    # Print the predicted class and probabilities
    print(f"Predicted Class: {class_names[predicted_class]}")
    print(f"Probabilities: {probabilities}")


In [7]:
image_path = 'BrainTumor_MRI/Testing/meningioma/Te-me_0073.jpg'
predict_image(image_path, model)

Predicted Class: meningioma
Probabilities: [0.20039243 0.38202462 0.22719419 0.19038871]


In [8]:
image_path = 'BrainTumor_MRI/Testing/pituitary/Te-pi_0024.jpg'
predict_image(image_path, model)

Predicted Class: notumor
Probabilities: [0.20336603 0.28343058 0.30817375 0.20502958]


In [9]:
image_path = 'BrainTumor_MRI/Testing/glioma/Te-gl_0081.jpg'
predict_image(image_path, model)

Predicted Class: meningioma
Probabilities: [0.19991969 0.38070673 0.22614908 0.19322444]


In [10]:
image_path = 'BrainTumor_MRI/Testing/notumor/Te-no_0097.jpg'
predict_image(image_path, model)

Predicted Class: notumor
Probabilities: [0.17920803 0.18779007 0.4546532  0.17834866]
